In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [2]:
config = {
    'project': 'babble',
    'domain': 'cdr',
#     'db_name': 'babble_protein_temp4',
    'splits': [0,1],
    'gold_explanations': True,
    'supervision': 'soft_majority',
    'gen_model_search_space': 1,
#     'gen_params_default': {
#         'epochs': 0
#     },
    'tune_b': False,
    'disc_model_class': 'logreg',
    'disc_model_search_space': 1,
}

In [3]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_cdr.db


In [4]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

Exception: spaCy not installed. Use `pip install spacy`.

In [ ]:
# from snorkel.annotations import load_gold_labels, load_label_matrix
# from scipy.sparse import save_npz
# TRAIN = 0
# DEV = 1

# L_train = load_label_matrix(pipe.session, split=TRAIN)
# save_npz('L_train_' + config['domain'], L_train)

# L_dev = load_label_matrix(pipe.session, split=DEV)
# save_npz('L_dev_' + config['domain'], L_dev)

# L_gold_dev = load_gold_labels(pipe.session, annotator_name='gold', split=DEV)
# save_npz('L_gold_dev_' + config['domain'], L_gold_dev)

In [ ]:
# %time pipe.parse()

In [ ]:
# %time pipe.extract()

In [ ]:
# for split in [0,1,2]:
#     print(session.query(pipe.candidate_class).filter(
#         pipe.candidate_class.split == split).count())

In [ ]:
# %time pipe.load_gold()

In [ ]:
# %time pipe.featurize()

In [ ]:
# %time pipe.collect()

In [ ]:
# log_forms = []
# for exp in pipe.explanations:
#     log_forms.append(str(exp.semantics))

In [ ]:
# import numpy as np
# towrite = np.array(log_forms)
# np.save("protein_lfs", towrite)

In [ ]:
# %time pipe.label()

In [ ]:
# from snorkel.annotations import load_gold_labels, load_label_matrix

# TEST = 2

# L_test = load_label_matrix(pipe.session, split=TEST)
# L_gold_test = load_gold_labels(pipe.session, annotator_name='gold', split=TEST)
# print(L_test.lf_stats(pipe.session, labels=L_gold_test))

In [ ]:
# %time pipe.supervise()

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist(pipe.train_marginals)
# plt.show()

### Attempt Metal

Load matrices

In [ ]:
# from snorkel.annotations import load_gold_labels, load_label_matrix

# TRAIN = 0
# DEV = 1

# L_train = load_label_matrix(pipe.session, split=TRAIN)

# L_dev = load_label_matrix(pipe.session, split=DEV)
# L_gold_dev = load_gold_labels(pipe.session, annotator_name='gold', split=DEV)

In [ ]:
# def convert_to_categorical(X):
#     """Convert {0,-1,1} -> {0,1,2}."""
#     X[X == 1] = 2
#     X[X == -1] = 1
#     return X

In [ ]:
# L_train = convert_to_categorical(L_train)
# L_dev = convert_to_categorical(L_dev)
# L_gold_dev = convert_to_categorical(L_gold_dev)

# # Fix for weird scipy.matrix format
# L_gold_dev = np.array(L_gold_dev.todense().T)[0]

In [ ]:
# from metal.class_hierarchy import ClassHierarchy

# task_to_lfs = {0 : range(L_train.shape[1])}
# ch = ClassHierarchy(L_train, task_to_lfs, [], [2])

# ch.grid_search_train(
#     L_dev,
#     L_gold_dev,
#     search_space={
#         'l2': [0.5], #[0.0, 0.1, 0.5, 1.0],
# #         'step_size': [0.01], #[0.01, 0.05, 0.1, 0.25]
#     },
#     score_metric='f1',
#     n_steps=500,
#     print_at=250
# )

# prec, rec, f1 = ch.f1_score(L_dev, L_gold_dev)
# print('DP on test set: P={} | R={} | F1={}'.format(prec, rec, f1))

In [ ]:
# plt.hist(ch.LF_accs(0))
# plt.show()

In [ ]:
# plt.hist(ch.conditional_probs(ch._parse_L(L_train), 0)[2])
# plt.show()

In [ ]:
# import random
# from scipy.sparse import csr_matrix, lil_matrix, issparse

# def add_pepper(L, p=0.1):
#     n, m = L.shape
#     polarity = np.zeros((m,))
#     for j in range(m):
#         polarity[j] = 1 if np.sum(L[:,j]) >= 0 else -1
    
#     L = lil_matrix(L)
#     for j in range(m):
#         opposite = polarity[j] * -1
#         for i in range(n):
#             if random.random() < p and not L[i,j]:
#                 L[i,j] = opposite
#     return csr_matrix(L)

# def to_metal_labels(L):
#     L[L == 1] = 2
#     L[L == -1] = 1
#     return L

# def transform_L(L_in, pepper=0):
#     if np.max(L_in) > 1:
#         raise Exception("L has already been transformed.")

#     if issparse(L_in):
#         L = csr_matrix(L_in.copy())
#     else:
#         L = L_in.copy()
        
#     if pepper:
#         L = add_pepper(L, p=pepper)
        
#     return to_metal_labels(L)

In [ ]:
# from metal.class_hierarchy import ClassHierarchy

# N_STEPS = 500

# task_to_lfs = { 0 : list(range(L_train.shape[1])) }
# gold_dev = transform_L(np.array(L_gold_dev.todense()).T[0])

# best = (0, 0, 0)
# for p in [0, 0.05, 0.1, 0.2, 0.3]:
#     L_t = transform_L(L_train, pepper=p)
#     L_d = transform_L(L_dev, pepper=0) # NOTE: p = 0 here
#     ch = ClassHierarchy(L_t, task_to_lfs, [], [2])
#     for l2 in [0.1, 0.25, 0.5, 1]:
#         print("p = {}, l2 = {}".format(p, l2))
#         ch.train(n_steps=N_STEPS, step_size=0.1, print_at=100, l2=l2, random_init=False)
#         pr, re, f1 = ch.f1_score(L_d, gold_dev)
#         print(pr, re, f1)
#         print("")
#         if f1 > best[2]:
#             best = (pr, re, f1)
#             opt_config = (p, l2)


# print("BEST MODEL:")
# print("p = {}, l2 = {}".format(*opt_config))
# print best
# p, l2 = opt_config
# L_t = transform_L(L_train, pepper=p)
# ch = ClassHierarchy(L_t, task_to_lfs, [], [2])
# ch.train(n_steps=N_STEPS, step_size=0.1, print_at=100, l2=l2, random_init=False)
# pr, re, f1 = ch.f1_score(L_d, gold_dev)

In [ ]:
# import matplotlib.pyplot as plt

# marginals = ch.conditional_probs(ch._parse_L(L_t), 0)[1]
# plt.hist(marginals)
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# from snorkel.contrib.pipelines.utils import score_marginals

# display = False
# for l2 in [0, 0.25, 0.5]:
#     ch.train(n_steps=1000, step_size=0.1, print_at=100, l2=l2)
#     marginals = ch.conditional_probs(ch._parse_L(L_train), 0)[1]
#     accuracies = ch.LF_accs(0)
#     dev_marginals = ch.conditional_probs(ch._parse_L(L_dev), 0)[1]
#     score_marginals(dev_marginals, L_gold_dev, b=0.5)
#     if display:
#         plt.hist(marginals)
#         plt.show()
#         plt.hist(accuracies)
#         plt.show()
#         plt.hist(dev_marginals)
#         plt.show()

In [ ]:
# marginals = ch.conditional_probs(ch._parse_L(L_train), 0)[2]
# pipe.train_marginals = marginals

### End Metal

In [ ]:
# pipe.gen_model.learned_lf_stats()['Accuracy']

In [ ]:
%time pipe.supervise()

In [ ]:
%time pipe.classify()

In [ ]:
w, b = pipe.disc_model.get_weights()
top_w = sorted(zip(w, range(len(w))), reverse=True)

In [ ]:
from snorkel.annotations import load_feature_matrix

X_train = load_feature_matrix(pipe.session, split=pipe.config['traditional_split'])

In [ ]:
for (w, idx) in top_w[:100]:
    f = X_train.get_key(pipe.session, idx)
    print(w, f)

### Prep for Chris

In [ ]:
# from scipy.sparse import save_npz

# L_train = load_label_matrix(pipe.session, split=TRAIN)
# save_npz('L_train', L_train)

# L_dev = load_label_matrix(pipe.session, split=DEV)
# save_npz('L_dev', L_dev)

# L_gold_dev = load_gold_labels(pipe.session, annotator_name='gold', split=DEV)
# save_npz('L_gold_dev', L_gold_dev)

In [ ]:
# m = L.shape[1]
# counts = np.zeros((m,m))
# X = np.zeros((m,m))
# arrays = [np.squeeze(L[:,i].toarray()) for i in range(L.shape[1])]
# foo = 0
# for i in range(m):
#     for j in range(i, m):
#         r = arrays[i]
#         t = arrays[j]
#         prod = np.multiply(r,t)
#         agree = sum(prod == 1)
#         disag = sum(prod == -1)
# #         print("{} {} {}".format(prod, agree, disag))
#         total = agree + disag
#         counts[i,j] = total
#         counts[i,j] = total
#         if i < j:
#             foo += disag
#         if not total:
#             X[i,j] = 0
#         else:
#             X[i,j] = 1.0/total * (agree - disag)
#             X[j,i] = 1.0/total * (agree - disag)

In [ ]:
# np.save('X', X)
# np.save('counts', counts)

In [ ]:
# def save_sparse_csr(filename, array):
#     np.savez(filename, data=array.data, indices=array.indices,
#              indptr=array.indptr, shape=array.shape)

# def load_sparse_csr(filename):
#     loader = np.load(filename)
#     return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
#                       shape=loader['shape'])

In [ ]:
# filename = "my_L"
# save_sparse_csr(filename, L)

In [ ]:
foo = ('.root', ('.label', ('.bool', True), ('.call', ('.in', 5))))

In [ ]:
foos = str(foo)

In [ ]:
bar = eval(foos)

In [ ]:
bar